This notebook is an implementation that starts with an Ising model that we got from other hardware (D-Wave Ocean SDK on their quantum annealer) and implements it as a QAOA on the IBM hardware with Qiskit.

In [ ]:
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.optimization.ising.common import sample_most_likely 
from qiskit.optimization.ising.max_cut import get_graph_solution 

from qiskit.quantum_info import Pauli
import numpy as np

# Make printing out some of the larger matrices a bit less painful
import pprint
pp = pprint.PrettyPrinter(indent=4)

# This we got from the Ocean SDK and copypasted here. Unfortunately the data types are not directly compatible.
ising = ({0: 4.0, 1: 5.0, 2: 3.5, 3: 5.75, 4: 2.75},
 {(0, 1): 5.25,
  (0, 2): 4.5,
  (0, 3): 5.25,
  (0, 4): 4.5,
  (1, 2): 4.5,
  (1, 3): 5.25,
  (1, 4): 4.5,
  (2, 3): 5.25,
  (2, 4): 5.25,
  (3, 4): 4.5},
 -27.75)

# We calculate the size of a matrix we will need to represent the Ising model above.
size = 0 
for x in ising[0]:
    size = x if x > size else size
size += 1    
print(size)

In [ ]:
# Now we dive into turning this simple ising object into a matrix that Qiskit tools including the QAOA can understand.
# Basically we need to indicate with the Z paulis which of the qubits this weight is applied to.
paulis = []
print("Empty:")
pp.pprint(paulis)
for x in ising[0]:
    xp = np.zeros(size, dtype=np.bool)
    zp = np.zeros(size, dtype=np.bool)

    zp[x] = True
    paulis.append([ising[0][x], Pauli(zp, xp)])
    
print("With diagonal elements:")
pp.pprint(paulis)

for x in ising[1]:
    xp = np.zeros(size, dtype=np.bool)
    zp = np.zeros(size, dtype=np.bool)

    zp[x[0]] = True
    zp[x[1]] = True
    paulis.append([ising[1][x], Pauli(zp, xp)])
print("With everything:")
pp.pprint(paulis)

In [ ]:
# We make an operator out of the matrix. In a way we just need to tell which qubits does the matrix sell apply to.
qubit_op = WeightedPauliOperator(paulis)

print("num qubits: {}".format(qubit_op.num_qubits))

pp.pprint(qubit_op.to_dict())

In [ ]:
# ========== CLASSICAL
# We use the ExactEigensolver to just solve the matrix without caring that they are binary numbers.

result = ExactEigensolver(qubit_op).run()
pp.pprint(result)
x = sample_most_likely(result['eigvecs'][0])
print(x)
print('energy:', result['energy'])
print('solution:', get_graph_solution(x))

In [ ]:
# ========== QUANTUM 
from qiskit.visualization import *
from qiskit.aqua import aqua_globals
aqua_globals.random_seed = 777 # This is random e.g. whatever you like

In [ ]:
# VQE parts
# VQE is the same as ExactEigensolver but a quantum version of the algorithm
from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua.components.optimizers import SPSA

var_form = RY(qubit_op.num_qubits, depth=7, entanglement='full')
optimizer = SPSA(max_trials=200)
vqe = VQE(qubit_op, var_form, optimizer)
backend = Aer.get_backend('qasm_simulator')
result = vqe.run(backend)

x = sample_most_likely(result['eigvecs'][0])
print(x)
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('solution:', get_graph_solution(x))

In [ ]:
plot_histogram(result['min_vector'])

In [ ]:
#QAOA parts
from qiskit import BasicAer, Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import SPSA, COBYLA, SLSQP

# We can choose from several classical optimizers
#optimizer = SPSA(max_trials=250)
optimizer = COBYLA(maxiter=2000) #1000
#optimizer = SLSQP(maxiter=1000)
qaoa = QAOA(qubit_op, optimizer, p=10)
backend = Aer.get_backend('qasm_simulator') 
qi = QuantumInstance(backend, shots=200)
result = qaoa.run(qi)
    
x = sample_most_likely(result['eigvecs'][0])
print(x)
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('solution:', get_graph_solution(x))

In [ ]:
plot_histogram(result['min_vector'])